### Programming in life sciences

[Anaconda](https://www.anaconda.com/download/success)

`conda create --name new_env`

`conda activate new_env`

Install libraries you want in isolated env

### Predicting open/close conformation of a kinase model



<b>Lecture 1.1: Introduction to alignments - exercise: align two homologous kinases (given as fasta files)</b>

Lecture 1.2: Introduction to superpositions - exercise: get RMSD of two structures

Lecture 2: Introduction to AlphaFold - exercise: get models from AlphaFold and classify them as open/close, do the same for models from Swiss-Model and AlphaFold-dropout

Lecture 3: Ligand docking (DiffDock) - exercise: get docking from DiffDock colab, find interacting atoms and visualize docking

### 1. Lecture: Needleman-Wunsch algorithm

In this exercise, we want to align two sequences obtained from fasta files

In [11]:
# numpy is a library providing a lot of numerical mathematical functionalities
# here we only use it to store stuff in matrices
import numpy as np

# we create a new class that will be used to define substitution matrices
class SubstitutionMatrix:
  """Read, store and analyse substitution matrices, default = BLOSUM62"""

  # this is the constructor function! When you create a new instance of type
  # Substitution Matrix, this function gets called. By default it reads in
  # the provided file: blosum62.txt... you can create your own matrix if
  # you like ;)
  def __init__(self, data_file: str = "blosum62.txt"): # takes datafile downloaded from adam
    """ Constructor function for the SubstitutionMatrix class
    Parameters
    ----------
    data_file : string
        Filename of the substitution matrix to read in, must contain 21 lines,
        with the first line containing the 20 amino acid one letter codes,
        by default reads the blosum62.txt matrix in the current folder
    """
    
    # read all the lines from the file at once  ('r' = read)
    with open(data_file,'r') as fh:
      data = fh.readlines()

    # the size of subsitution matrices must be 20x20
    # the first line of the file gives the amino acid one letter codes
    # which gives a total of 21 lines in the file
    if len(data) != 21:
      err = "Expected exactly 21 lines in matrix file!"
      err += " One line defining the amino acid one letter codes and "
      err += "20 lines defining the substitution matrix"
      raise RuntimeError(err)    

    # get the amino acid one letter codes
    self.one_letter_codes = data[0].strip()

    # check that 20 amino acids are defined
    if len(self.one_letter_codes) != 20:
      err = "First line must contain exactly 20 letters representing "
      err += "the amino acids!"
      raise RuntimeError(err)

    # initialize the substitution matrix with zeros
    self.matrix = np.zeros((20,20))

    # fill in the matrix by reading the file line by line
    # note that range returns a vector starting at 0 to the integer in argument - 1 
    for i in range(20):
      data_line = data[1+i].split()
      if len(data_line) != 20:
        raise RuntimeError("Each data line must contain exactly 20 elements!")
      for j in range(20):
        self.matrix[(i,j)] = int(data_line[j])


  # The function where you can extract the score for a specific amino acid substitution...
  # e.g. mat.get_score('W', 'A')
  def get_score(self, aa_one: str, aa_two: str):
    """ Extract the substitution score for two amino acids from the substitution matrix
    Parameters
    ----------
    aa_one : string
        One letter amino acid code
    aa_two : string
        One letter amino acid code
    Returns
    -------
    self.matrix[(idx_one,idx_two)] : float
        Substitution matrix entry for the substitution of amino acid one and amino acid two,
        note that the matrices are symmetric, so get_score(aa_one, aa_two) is equal to get_score(aa_two, aa_one)
    """

    # get the indices of the amino acids in the matrix
    idx_one = self.one_letter_codes.find(aa_one)
    idx_two = self.one_letter_codes.find(aa_two)

    if idx_one == -1:
      raise RuntimeError("aa_one must be one of: " + self.one_letter_codes)
    
    if idx_two == -1:
      raise RuntimeError("aa_two must be one of: " + self.one_letter_codes)

    return self.matrix[(idx_one,idx_two)]


In [12]:
# Decorating functions with docstrings is not only best practice,
# it also helps automated documentation generation with tools
# like Sphinx (https://www.sphinx-doc.org) or simply get more
# information using the Python help function:
help(SubstitutionMatrix.get_score)

Help on function get_score in module __main__:

get_score(self, aa_one: str, aa_two: str)
    Extract the substitution score for two amino acids from the substitution matrix
    Parameters
    ----------
    aa_one : string
        One letter amino acid code
    aa_two : string
        One letter amino acid code
    Returns
    -------
    self.matrix[(idx_one,idx_two)] : float
        Substitution matrix entry for the substitution of amino acid one and amino acid two,
        note that the matrices are symmetric, so get_score(aa_one, aa_two) is equal to get_score(aa_two, aa_one)



In [13]:
# Construct a default substitution matrix
subst_matrix = SubstitutionMatrix()

# get score of substituting ASP with ASN and GLY with PRO
print("Score D->N:", subst_matrix.get_score('N', 'D'))
print("Score G->P:", subst_matrix.get_score('G', 'P'))

Score D->N: 1.0
Score G->P: -2.0


In [14]:
def ReadFasta(filename: str):
    """ A simplistic reader that only checks if the first line starts with '>' and assumes that the whole rest is the input sequence
    Parameters
    ----------
    filename : string
        Filename of the FASTA file to read
    Returns
    -------
    s : string
        Sequence read from the input FASTA file
    """
    # TODO Fill the function

    f = open(filename)
    s = ''

    first_line = f.readline().strip()

    if first_line.startswith(">"):
        for line in f: 
            s += line.strip()
    
    else: 
        print("Error: File doesn't seem to be Fasta, or just doesnt start with '>'. ")
        s += ''

    f.close()
    return s

    pass

In [15]:
# TODO
# read in the two sequences to align s1 & s2
# the two files are located in the same directory as the notebook

s1 = ReadFasta("s1.fasta")
s2 = ReadFasta("s2.fasta")

print(s1)
print(s2)

VDRVG
VERMAG


#### Write the code for running the Needleman-Wunsch algorithm

In [16]:
# define a penalty for introducing a gap in the sequence alignment
gap_penalty = -8

Setup scoring and backtracking matrix

In [17]:
# Keeps track of what path we chose going through the matrix

# s1 = ReadFasta("s1.fasta")
# s2 = ReadFasta("s2.fasta")

s1 = ReadFasta("Q47XA8.fasta")
s2 = ReadFasta("P69441.fasta")

n_rows = len(s1)+1 # s1 goes from top to bottom (in the algorith we access i-1, so we need to add 1)
n_cols = len(s2)+1 # s2 goes from left to right

scoring_matrix = np.zeros((n_rows, n_cols))

# backtrack encoding:
# 1 => aligned
# 2 => deletion in s1 (insertion in s2 respectively)
# 3 => deletion in s2 (insertion in s1 respectively)
backtrack_matrix = np.zeros((n_rows, n_cols))

# the first row and the first column can already be prefilled
for i in range(1, n_rows):
    scoring_matrix[(i,0)] = i*gap_penalty
    backtrack_matrix[(i,0)] = 3
for i in range(1, n_cols):
    scoring_matrix[(0,i)] = i*gap_penalty
    backtrack_matrix[(0,i)] = 2

print("row sequence: ", s1)
print("col sequence: ", s2)
print("empty scoring matrix: ")
print(scoring_matrix)
print("empty backtracking matrix: ")
print(backtrack_matrix)

row sequence:  MRIVLLGAPGAGKGTQAQFLMAKFGIPQISTGDMLRAAIKAGSELGNKAKAVMDAGQLVSDDLIIGLVKERVAQEDCKAGFLLDGFPRTIPQADAMKESGIVVDHVLEFDVPDEVIVERMAGRRVHSGSGRVYHLVYNPPKVEGKDDVSGDDLSIRPDDEEATVRKRLAIYHEQTKPLVDFYQAEAKSGSCSYLTIDGTQAVEKVNQLLSAQLA
col sequence:  MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG
empty scoring matrix: 
[[    0.    -8.   -16. ... -1696. -1704. -1712.]
 [   -8.     0.     0. ...     0.     0.     0.]
 [  -16.     0.     0. ...     0.     0.     0.]
 ...
 [-1696.     0.     0. ...     0.     0.     0.]
 [-1704.     0.     0. ...     0.     0.     0.]
 [-1712.     0.     0. ...     0.     0.     0.]]
empty backtracking matrix: 
[[0. 2. 2. ... 2. 2. 2.]
 [3. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]
 ...
 [3. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]


#### Fill scoring and backtracking matrices

Every position in the scoring matrix represents the best local solution given the
steps I can take
1. Align two residues => Move along the diagonal (consume a letter from s1 and s2),
                        the value of the scoring matrix to the upper left plus the corresponding score from the scoring matrix
2. Apply deletion/gap in s1 => Move right in the scoring matrix (consume a letter from s2), 
                              the value from the scoring matrix to the left plus the penalty for a gap
3. Apply deletion/gap in s2 => Move down in the scoring matrix (consume a letter from s1),
                              the value from the scoring matrix above plus the penalty for a gap

The goal is to identify the best scoring solution from those three,
set the according score in the scoring matrix and keep track of this local
step in the backtracking matrix. 

The backtracking matrix stores the path I took, e.g. 1 for the first option
(see backtracking encoding in previous code cell)

In [18]:
for r_idx in range(1, n_rows):
    for c_idx in range(1, n_cols):

        # TODO calculate the scores


        score_match = scoring_matrix[(r_idx - 1, c_idx - 1)] + subst_matrix.get_score(s1[r_idx-1], s2[c_idx-1])

        score_del_1 = scoring_matrix[(r_idx, c_idx - 1)] + gap_penalty
        score_del_2 = scoring_matrix[(r_idx - 1, c_idx)] + gap_penalty
        
        # TODO Fill the values at position (r_idx, c_idx) in scoring_matrix and backtracking_matrix
        # you can assign an element with
        # scoring_matrix[(r_idx, c_idx)] = my_awesome_value
        if score_match >= score_del_1 and score_match >= score_del_2: 
            scoring_matrix[(r_idx, c_idx)] = score_match #fill score into the matrix
            backtrack_matrix[(r_idx, c_idx)] = 1 #fill with corresponding number 
        elif score_del_1 >= score_match and score_del_1 >= score_del_2:
            scoring_matrix[(r_idx, c_idx)] = score_del_1
            backtrack_matrix[(r_idx, c_idx)] = 2
        else:
            scoring_matrix[(r_idx, c_idx)] = score_del_2
            backtrack_matrix[(r_idx, c_idx)] = 3
        
print("filled scoring matrix:")
print(scoring_matrix)
print("filled backtracking matrix:")
print(backtrack_matrix)

filled scoring matrix:
[[    0.    -8.   -16. ... -1696. -1704. -1712.]
 [   -8.     5.    -3. ... -1683. -1691. -1699.]
 [  -16.    -3.    10. ... -1670. -1678. -1686.]
 ...
 [-1696. -1683. -1670. ...   785.   780.   772.]
 [-1704. -1691. -1678. ...   784.   789.   781.]
 [-1712. -1699. -1686. ...   776.   783.   789.]]
filled backtracking matrix:
[[0. 2. 2. ... 2. 2. 2.]
 [3. 1. 2. ... 2. 2. 2.]
 [3. 3. 1. ... 2. 2. 2.]
 ...
 [3. 3. 3. ... 1. 1. 1.]
 [3. 3. 3. ... 1. 1. 2.]
 [3. 3. 3. ... 3. 1. 1.]]


#### Perform backtracking to get final alignment

Remember the encoding:

1 => aligned, move along the diagonal

2 => deletion in s1 (insertion in s2 respectively), move right

3 => deletion in s2 (insertion in s1 respectively), move down

In [19]:
# In principle we start at the bottom right of the backtracking matrix and
# work our way through the matrix until we hit the upper left

r_idx = n_rows-1
c_idx = n_cols-1
path = list()

# if we hit zero, we're at the upper left corner and can stop
while backtrack_matrix[(r_idx, c_idx)] != 0:
    path.append(backtrack_matrix[(r_idx, c_idx)])
    if backtrack_matrix[(r_idx, c_idx)] == 1: 
        c_idx = c_idx - 1
        r_idx = r_idx - 1
        
    elif backtrack_matrix[(r_idx, c_idx)] == 2:
        c_idx = c_idx - 1

    elif backtrack_matrix[(r_idx, c_idx)] == 3:
        r_idx = r_idx - 1
        
# backtracking comes from the back, so lets reverse...
path.reverse()

aln_s1 = []
aln_s2 = []
s1_idx = 0
s2_idx = 0

for p in path:
    if p == 1:
        aln_s1.append(s1[s1_idx])
        aln_s2.append(s2[s2_idx])
        s1_idx += 1
        s2_idx += 1
    elif p == 2:
        aln_s1.append('-')
        aln_s2.append(s2[s2_idx])
        s2_idx += 1
    else:
        aln_s1.append(s1[s1_idx])
        aln_s2.append('-')
        s1_idx += 1
        
aln_s1 = ''.join(aln_s1)
aln_s2 = ''.join(aln_s2)
        
print("input sequence s1:", s1)
print("input sequence s2:", s2)
print("\noptimal alignment:")
print("s1:", aln_s1)
print("s2:", aln_s2)


input sequence s1: MRIVLLGAPGAGKGTQAQFLMAKFGIPQISTGDMLRAAIKAGSELGNKAKAVMDAGQLVSDDLIIGLVKERVAQEDCKAGFLLDGFPRTIPQADAMKESGIVVDHVLEFDVPDEVIVERMAGRRVHSGSGRVYHLVYNPPKVEGKDDVSGDDLSIRPDDEEATVRKRLAIYHEQTKPLVDFYQAEAKSGSCSYLTIDGTQAVEKVNQLLSAQLA
input sequence s2: MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG

optimal alignment:
s1: MRIVLLGAPGAGKGTQAQFLMAKFGIPQISTGDMLRAAIKAGSELGNKAKAVMDAGQLVSDDLIIGLVKERVAQEDCKAGFLLDGFPRTIPQADAMKESGIVVDHVLEFDVPDEVIVERMAGRRVHSGSGRVYHLVYNPPKVEGKDDVSGDDLSIRPDDEEATVRKRLAIYHEQTKPLVDFYQAEAKSGSCSYLTIDGTQAVEKVNQLLSAQLA
s2: MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG


In [9]:
# let's put everything in one function
def Align(s1: str, s2: str):
    import numpy as np
    """ Alignment of two strings of amino acids according to the Needleman-Wunsch algorithm
    Parameters
    ----------
    s1 : string
        First amino acid string to align
    s2 : string
        Second amino acid string to align
    Returns
    -------
    aln_s1 : string
        First amino acid string, aligned
    aln_s2 : string
        Second amino acid string, aligned
    """
    
    # Are we meant to copy paste all code so far into here? 

    subst_matrix = SubstitutionMatrix() # redefine / create new one

    # Write the code for running the Needleman-Wunsch algorithm
    
    gap_penalty = -8

    n_rows = len(s1)+1 # s1 goes from top to bottom (in the algorith we access i-1, so we need to add 1)
    n_cols = len(s2)+1

    scoring_matrix = np.zeros((n_rows, n_cols))

    # backtrack encoding:
    # 1 => aligned
    # 2 => deletion in s1 (insertion in s2 respectively)
    # 3 => deletion in s2 (insertion in s1 respectively)
    backtrack_matrix = np.zeros((n_rows, n_cols))

    # the first row and the first column can already be prefilled
    for i in range(1, n_rows):
        scoring_matrix[(i,0)] = i*gap_penalty
        backtrack_matrix[(i,0)] = 3
    for i in range(1, n_cols):
        scoring_matrix[(0,i)] = i*gap_penalty
        backtrack_matrix[(0,i)] = 2

    # From textblock #### Fill scoring and backtracking matrices
    
    for r_idx in range(1, n_rows):
        for c_idx in range(1, n_cols):

            score_match = scoring_matrix[(r_idx - 1, c_idx - 1)] + subst_matrix.get_score(s1[r_idx-1], s2[c_idx-1])

            score_del_1 = scoring_matrix[(r_idx, c_idx - 1)] + gap_penalty
            score_del_2 = scoring_matrix[(r_idx - 1, c_idx)] + gap_penalty
            
            # TODO Fill the values at position (r_idx, c_idx) in scoring_matrix and backtracking_matrix
            # you can assign an element with
            # scoring_matrix[(r_idx, c_idx)] = my_awesome_value
            if score_match >= score_del_1 and score_match >= score_del_2: 
                scoring_matrix[(r_idx, c_idx)] = score_match #fill score into the matrix
                backtrack_matrix[(r_idx, c_idx)] = 1 #fill with corresponding number 
            elif score_del_1 >= score_match and score_del_1 >= score_del_2:
                scoring_matrix[(r_idx, c_idx)] = score_del_1
                backtrack_matrix[(r_idx, c_idx)] = 2
            else:
                scoring_matrix[(r_idx, c_idx)] = score_del_2
                backtrack_matrix[(r_idx, c_idx)] = 3


    # From textblock #### Perform backtracking to get final alignment
    
    r_idx = n_rows-1
    c_idx = n_cols-1
    path = list()

    # if we hit zero, we're at the upper left corner and can stop
    while backtrack_matrix[(r_idx, c_idx)] != 0:
        path.append(backtrack_matrix[(r_idx, c_idx)])
        if backtrack_matrix[(r_idx, c_idx)] == 1: 
            c_idx = c_idx - 1
            r_idx = r_idx - 1
            
        elif backtrack_matrix[(r_idx, c_idx)] == 2:
            c_idx = c_idx - 1

        elif backtrack_matrix[(r_idx, c_idx)] == 3:
            r_idx = r_idx - 1
            
    # backtracking comes from the back, so lets reverse...
    path.reverse()

    aln_s1 = []
    aln_s2 = []
    s1_idx = 0
    s2_idx = 0

    for p in path:
        if p == 1:
            aln_s1.append(s1[s1_idx])
            aln_s2.append(s2[s2_idx])
            s1_idx += 1
            s2_idx += 1
        elif p == 2:
            aln_s1.append('-')
            aln_s2.append(s2[s2_idx])
            s2_idx += 1
        else:
            aln_s1.append(s1[s1_idx])
            aln_s2.append('-')
            s1_idx += 1
            
    aln_s1 = ''.join(aln_s1)
    aln_s2 = ''.join(aln_s2)

    # print("Input sequence 1: ", s1)
    # print("Input sequence 2: ", s2)
    # print("Aligned sequence 1: ", aln_s1)
    # print("Aligned sequence 2: ", aln_s2)


    return aln_s1, aln_s2


alignments = Align("VDRVG", "VERMAG")
print(alignments[0])
print(alignments[1])


VDRV-G
VERMAG


In [20]:
# Read two fasta files of bacterial kinase homologs (Q47XA8 & P69441) and get the alignment\

subst_matrix = SubstitutionMatrix()

bac1_kinase = ReadFasta("Q47XA8.fasta")
bac2_kinase = ReadFasta("P69441.fasta")

print(bac1_kinase)
print(bac2_kinase)

#print(bac1_kinase)

# TODO get alignment
alignments = Align(bac1_kinase, bac2_kinase)
bac1_aln = alignments[0]
bac2_aln = alignments[1]

print(bac1_aln)
print(bac2_aln)

print(backtrack_matrix)


MRIVLLGAPGAGKGTQAQFLMAKFGIPQISTGDMLRAAIKAGSELGNKAKAVMDAGQLVSDDLIIGLVKERVAQEDCKAGFLLDGFPRTIPQADAMKESGIVVDHVLEFDVPDEVIVERMAGRRVHSGSGRVYHLVYNPPKVEGKDDVSGDDLSIRPDDEEATVRKRLAIYHEQTKPLVDFYQAEAKSGSCSYLTIDGTQAVEKVNQLLSAQLA
MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG
MRIVLLGAPGAGKGTQAQFLMAKFGIPQISTGDMLRAAIKAGSELGNKAKAVMDAGQLVSDDLIIGLVKERVAQEDCKAGFLLDGFPRTIPQADAMKESGIVVDHVLEFDVPDEVIVERMAGRRVHSGSGRVYHLVYNPPKVEGKDDVSGDDLSIRPDDEEATVRKRLAIYHEQTKPLVDFYQAEAKSGSCSYLTIDGTQAVEKVNQLLSAQLA
MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG
[[0. 2. 2. ... 2. 2. 2.]
 [3. 1. 2. ... 2. 2. 2.]
 [3. 3. 1. ... 2. 2. 2.]
 ...
 [3. 3. 3. ... 1. 1. 1.]
 [3. 3. 3. ... 1. 1. 2.]
 [3. 3. 3.

In [13]:
# Sequence identity is a simple proxy for evolutionary distance
def SeqID(aln_s1: str, aln_s2: str):
    """ Computes sequence identity - the percentage of conserved
    amino acids
    ----------
    aln_s1 : string
        First aligned amino acid string
    aln_s2 : string
        Second aligned amino acid string
    Returns
    -------
    seqid : float
        The sequence identity
    """
    # must have same length
    assert(len(aln_s1) == len(aln_s2))
    n_match = 0
    n_aligned = 0
    for a, b in zip(aln_s1, aln_s2):
        if a!='-' and b!='-':
            n_aligned += 1
            if a == b:
                n_match += 1
    if n_aligned != 0:
        # explicitely cast to float here
        # This is not required in Python3 but Python2 would perform
        # an integer division otherwise
        return float(n_match)/n_aligned*100
    return 0.0

In [14]:
# Python provides several ways of string formatting
# one example are so called f-strings
print(f"SeqID: {SeqID(bac1_aln, bac2_aln):.2f}%")

SeqID: 0.00%
